### 6.2.1 互相关运算

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
d2l.corr2d

<function d2l.torch.corr2d(X, K)>

In [3]:
X = torch.tensor([
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8],
], dtype=torch.float32)
K = torch.tensor([
    [0, 1],
    [2, 3],
], dtype=torch.float32)
d2l.corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

### 6.2.2 卷积层

In [4]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()

        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return d2l.conv2d(x, self.weight, self.bias)

### 6.2.3 图像中目标的边缘检测

In [5]:
X = torch.ones([6, 8])
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [6]:
K = torch.tensor([[1.0, -1.0]])

In [7]:
Y = d2l.corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [8]:
d2l.corr2d(X.T, K)  # 只可以检测垂直边缘, 无法检测水平边缘

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

### 6.2.4 学习卷积核

In [12]:
conv2d = nn.Conv2d(1, 1, kernel_size=[1, 2], bias=False)

X = X.reshape([1, 1, 6, 8])
Y = Y.reshape([1, 1, 6, 7])
lr = 3e-2

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= lr * conv2d.weight.grad

    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

epoch 2, loss 6.059
epoch 4, loss 1.609
epoch 6, loss 0.513
epoch 8, loss 0.185
epoch 10, loss 0.072


In [13]:
conv2d.weight.data.reshape([1, 2])

tensor([[ 1.0172, -0.9630]])